# Assignment 1 - In-Context Learning

In this assignment, students experiment with in-context learning by selecting and ordering demonstrations to train a large language model at inference time to classify text. In this task, an online store is interested in classifying whether a review describes one or more general topics of interest. The topics are specific to a class of product, in this case vacuum cleaners. Other topics would be relevant to other products.

The dataset has been divided into a development, training and test sets. Students should practice setting up their experiments and writing their prompts using only the development set. Demonstrations for in-context leanring can be drawn from the training set. Final evaluation prior to submission should use the test set.

In [1]:
from openai import OpenAI
client = OpenAI()

def prompt_model(prompt):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        store=True,
        messages=[
            {"role": "user", 'content': prompt}
        ]
    )
    return completion.choices[0].message.content

## Open Source Models (Optional)

If students wish to evaluate their solution on open source models, they may use Ollama, if their hardware supports it.

In [2]:
from ollama import chat
from ollama import ChatResponse

def prompt_ollama(prompt):
    response: ChatResponse = chat(model='llama3.3', messages=[{
        'role': 'user',
        'content': prompt,
      },
    ])
    return response['message']['content']

## Load Reviews with Hashtags

The dataset is partitioned into development, training and testing sets. While writing the code to setup your experiments and write your prompts, only use the development set. The training set should be used to sample demonstrations. Only when your code is completed and you are ready to turn in your assignment should you run your experiment on the test set.

In [3]:
import json

data_dev = json.load(open('dataset-dev.json', 'r'))
data_train = json.load(open('dataset-train.json', 'r'))
data_test = json.load(open('dataset-test.json', 'r'))

print('\nDataset Sizes: Dev %i, Train %i, Test %i\n' % (len(data_dev), len(data_train), len(data_test)))

data_dev[0]


Dataset Sizes: Dev 100, Train 100, Test 300



{'text': "Lightweight and decent suction. Wheels don't glide great but loosening up. Looks like a toy! Lol colors are very bright!",
 'tags': ['#DesignAndUsabilityIssues', '#CustomerExperienceAndExpectations']}

## Define the Hashtag List for Prediction

In [4]:
tags = [
    '#DesignAndUsabilityIssues',
    '#PerformanceAndFunctionality',
    '#BatteryAndPowerIssues',
    '#DurabilityAndMaterialConcerns',
    '#MaintenanceAndCleaning',
    '#CustomerExperienceAndExpectations',
    '#ValueForMoneyAndInvestment',
    '#AssemblyAndSetup'
]

tag_list = ' '.join(tags)

## Review the Hashtag Distribution

In general, it is good practice when classifying items to know the distribution of target categories. Categories that are underrepresented, especially in the training data, would lead to underperformance.

## Define the Prompt and Experiment

The experiment generally has the following steps: (1) sample the training data to identify k demonstrations for 0 =< k < training set size; (2) construct linearize the demonstrations into text; (3) iterate over the test data and insert the test review and text linearization of the demonstrations into the prompt template; (4) send the prompt to the model and receive the response; (5) validate the response, if the response passes then store the response for later, else if the response fails validation, then save the response to a list of errors. It is generally good to save responses and errors with an index that can be linked back to the test data.

After running the experiment, the evaluation metrics should be computed from the answers and the errors should be inspected. Adjustments to the prompt and/or experiment can be made to reduce the errors, e.g., by post-processing the responses prior to validation.

## Evaluate the Experimental Results

The evaluation metrics include precision, recall and F1 score. For the total number of true positives (tp), false positives (fp) and false negatives (fn), these calculations should be used to report results:
* Precision = tp / (tp + fp)
* Recall = tp / (tp + fn)
* F1 = 2tp / (2tp + fp + fn)